# Proyecto 1
* Flavio Galán - 22386
* Josue Say - 22801
* Isabella Miralles

## Descarga de los Datos

Para descargar los datos se utiliza Selenium y Python, además se tiene un sistema de cacheo que funciona en base al archivo `links_cache.txt`. Si este archivo ya existe entonces no se realizará ninguna descarga de los datos. Los datos se guardan en varios archivos txt. Cada uno representa la columna del dataframe y todos los datos de esa columna. No se tienen en un formato JSON ni CSV sino que se guardan directamente en formato python para la facilidad de extracción usando el mismo lenguaje. Ya que estos son los datos sucios no hay ningún problema con guardarlos así, los datos limpios ya se guardarán en un formaton más estandarizado.

### Importaciones y configuración para el web scraping


In [1159]:
import os
import time
# import subprocess (se reemplazo con el uso de os para cualquier sistema operativo)
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.select import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import numpy as np

### Configuración de rutas y archivo de caché

In [1160]:
cache_file_name = "./cache/links_cache.txt"
df_cache_file = "./cache/dataframe_cache.txt"
zip_dir = "zips/"
files_dir = "data/"
csv_file_path = "./data/data_unified.csv"
reports = "reportes/"


os.makedirs("cache", exist_ok=True)
os.makedirs("data", exist_ok=True)
os.makedirs("zips", exist_ok=True)
os.makedirs("reportes", exist_ok=True)

### Inicialización de estructuras para almacenar los datos

In [1161]:
codigo = []
distrito = []
departamento = []
municipio = []
establecimiento = []
direccion = []
telefono = []
supervisor = []
director = []
nivel = []
sector = []
area = []
status = []
modalidad = []
jornada = []
plan = []
departamental = []
agggrArrays = [
    codigo,
    distrito,
    departamento,
    municipio,
    establecimiento,
    direccion,
    telefono,
    supervisor,
    director,
    nivel,
    sector,
    area,
    status,
    modalidad,
    jornada,
    plan,
    departamental,
]
filenames = [
    "codigo",
    "distrito",
    "departamento",
    "municipio",
    "establecimiento",
    "direccion",
    "telefono",
    "supervisor",
    "director",
    "nivel",
    "sector",
    "area",
    "status",
    "modalidad",
    "jornada",
    "plan",
    "departamental",
]

### Carga de datos (web scrapping)

In [1162]:
if not os.path.exists(cache_file_name):
    with webdriver.Firefox() as driver:
        driver.implicitly_wait(3)

        driver.get("http://www.mineduc.gob.gt/BUSCAESTABLECIMIENTO_GE/")
        assert "Búsqueda de centros" in driver.title

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, "//*[@id='_ctl0_ContentPlaceHolder1_cmbDepartamento']")
            )
        )
        deptSelect = driver.find_element(
            By.XPATH, "//*[@id='_ctl0_ContentPlaceHolder1_cmbDepartamento']"
        )
        deptSelect = Select(deptSelect)
        depts = deptSelect.options
        print("Getting depts")
        for idx in range(len(depts) - 1):
            print("Selecting dept", idx + 1)
            deptSelect = driver.find_element(
                By.XPATH, "//*[@id='_ctl0_ContentPlaceHolder1_cmbDepartamento']"
            )
            deptSelect = Select(deptSelect)
            deptSelect.select_by_index(idx + 1)
            print("Finding education level")
            levelSelect = driver.find_element(
                By.XPATH, '//*[@id="_ctl0_ContentPlaceHolder1_cmbNivel"]'
            )
            levelSelect = Select(levelSelect)
            print("Selecting diversificado")
            levelSelect.select_by_value("46")  # 46 es Diversificado
            print("Finding search button")
            btn = driver.find_element(
                By.XPATH, "//*[@id='_ctl0_ContentPlaceHolder1_IbtnConsultar']"
            )
            btn.click()
            print("Clicking button")

            # Wait for results
            print("Esperando por resultados")
            time.sleep(5)
            print("Asumimos que se obtuvieron los resultados")

            table = driver.find_element(
                By.XPATH, "//*[@id='_ctl0_ContentPlaceHolder1_dgResultado']"
            )
            rows = table.find_elements(By.XPATH, ".//tr")
            for rowIdx in range(
                1, len(rows) - 1
            ):  # La última fila siempre es una vacía
                cells = rows[rowIdx].find_elements(By.XPATH, ".//td")

                for cellIdx in range(1, len(cells)):
                    agggrArrays[cellIdx - 1].append(
                        cells[cellIdx].get_attribute("textContent")
                    )

            # print("Obtained following names")
            # print(establecimiento)
            # exit(1)

        print("Saving cache...")
        os.makedirs(zip_dir, exist_ok=True)
        for idx in range(len(filenames)):
            filename = filenames[idx]
            data = agggrArrays[idx]

            lines = ["["]
            for val in data:
                line = f"\t'''{val}''',\n"
                lines.append(line)
            lines.append("]")

            with open(zip_dir + filename + ".txt", "w") as file:
                file.writelines(lines)

        os.makedirs(os.path.dirname(cache_file_name), exist_ok=True)
        with open(cache_file_name, "w") as file:
            file.write("DELETE ME IF YOU WANT TO REDOWNLOAD DATA!")


### Carga de datos al DataFrame

Este bloque verifica si existen los archivos de caché (`links_cache.txt` y `dataframe_cache.txt`). Si no hay caché, construye el `DataFrame` desde los datos extraídos y guarda un `.csv`. Si los cachés ya existen, carga directamente el `DataFrame` desde el archivo CSV, evitando repetir procesos.

In [1163]:
def loadDfCache(
    filenames=None,
    agggrArrays=None,
    csvFile=csv_file_path,
    dfCache=df_cache_file
):
    if os.path.exists(csvFile):
        df = pd.read_csv(csvFile, encoding="utf-8-sig")
        print("DataFrame loaded from CSV:")
        print(df)

        if not os.path.exists(dfCache):
            with open(dfCache, "w") as f:
                f.write("DataFrame cache created.")
    else:
        df = pd.DataFrame(
            {colName: colData for (colName, colData) in zip(filenames, agggrArrays)}
        )
        print("The resulting DataFrame is:")
        print(df)

        df.to_csv(csvFile, index=False, encoding="utf-8-sig")

        with open(dfCache, "w") as f:
            f.write("DataFrame cache created.")

    return df

In [1164]:
df = loadDfCache(filenames, agggrArrays)

DataFrame loaded from CSV:
             codigo distrito  departamento  municipio  \
0     16-01-0138-46   16-031  ALTA VERAPAZ      COBAN   
1     16-01-0139-46   16-031  ALTA VERAPAZ      COBAN   
2     16-01-0140-46   16-031  ALTA VERAPAZ      COBAN   
3     16-01-0141-46   16-005  ALTA VERAPAZ      COBAN   
4     16-01-0142-46   16-005  ALTA VERAPAZ      COBAN   
...             ...      ...           ...        ...   
6594  19-09-0040-46   19-021        ZACAPA   LA UNION   
6595  19-09-0048-46   19-021        ZACAPA   LA UNION   
6596  19-10-0013-46   19-015        ZACAPA      HUITE   
6597  19-10-1009-46   19-015        ZACAPA      HUITE   
6598  19-11-0018-46   19-020        ZACAPA  SAN JORGE   

                                        establecimiento  \
0                                        COLEGIO  COBAN   
1                     COLEGIO PARTICULAR MIXTO  VERAPAZ   
2                               COLEGIO "LA INMACULADA"   
3              ESCUELA NACIONAL DE CIENCIAS COMERCIA

## Estructura del Conjunto de Datos Crudo

En esta etapa se realiza un análisis exploratorio preliminar del conjunto de datos descargado, con el objetivo de conocer su estructura general. Se identifica la cantidad de filas y columnas, la presencia de datos duplicados, valores nulos por variable, y los tipos de datos registrados junto con la primera limpieza para detectar los criterios mencionados.

In [1165]:
def generateDataReport(df: pd.DataFrame, saveToFile: bool = True, filename: str = "reporte_general"):
    """
    Genera un resumen general del DataFrame con limpieza de datos vacíos o invisibles.

    Parámetros:
    - df: DataFrame de entrada.
    - saveToFile: Si es True, guarda el reporte en 'reportes/{filename}.txt'.
    - filename: Nombre del archivo de texto (sin extensión).

    Retorna:
    - Tuple: (lista del reporte como strings, DataFrame limpio)
    """

    # Limpieza profunda sin sobreescribir np.nan
    df_clean = df.copy()

    def clean_value(val):
        if pd.isna(val):
            return np.nan
        val = str(val).replace("\xa0", "").strip()
        return val if val != "" else np.nan

    for col in df_clean.select_dtypes(include=["object"]).columns:
        df_clean[col] = df_clean[col].apply(clean_value)

    if saveToFile:
        os.makedirs("reportes", exist_ok=True)
        file_path = os.path.join("reportes", f"{filename}.txt")

    num_filas, num_columnas = df_clean.shape
    duplicados_filas = df_clean.duplicated().sum()
    nulos_por_columna = df_clean.isnull().sum()
    tipos_datos = df_clean.dtypes

    # Crear el reporte
    reporte = []
    reporte.append(f"Total de filas: {num_filas}")
    reporte.append(f"Total de columnas: {num_columnas}")
    reporte.append(f"Filas duplicadas: {duplicados_filas}")
    reporte.append("\nValores nulos por columna:")
    reporte.extend([f"{col}: {nulos}" for col, nulos in nulos_por_columna.items()])
    reporte.append("\nTipos de datos por columna:")
    reporte.extend([f"{col}: {tipo}" for col, tipo in tipos_datos.items()])

    if saveToFile:
        with open(file_path, "w", encoding="utf-8") as f:
            f.write("\n".join(reporte))
        print(f"Reporte generado en '{file_path}'")

    return reporte, df_clean

In [1166]:
is_generate_report = True
reporte, df_clean = generateDataReport(df, saveToFile=is_generate_report, filename="reporte_data_v0")
df_clean.to_csv("./data/data_clean_v0.csv", index=False, encoding="utf-8")
# print("\n".join(report_lines))

Reporte generado en 'reportes\reporte_data_v0.txt'


> Nota: Se puede modificar la bandera para omitir la creación del reporte inicial.

## Limpieza de los datos
Se procede a ejecutar las transformaciones previamente ideadas y a unificar todos los datasets en uno solo.

### Importaciones y configuración

In [1167]:
import re
import pandas as pd
import unicodedata
import numpy as np
import csv

### Establecimientos

Primero normalizamos los textos eliminando comillas, tildes y espacios extra, dejando todo en mayúsculas. Luego, con base en reglas y excepciones definidas, identifica el tipo de establecimiento (como COLEGIO, INSTITUTO, ESCUELA, etc.) y lo guarda en una nueva columna. Finalmente, genera archivos con los establecimientos agrupados por tipo, listos para análisis o reporte.

In [1168]:
def cleanTextColumn(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    df = df.copy()

    # Limpieza básica
    df[column_name] = (
        df[column_name]
        .astype(str)
        .str.replace(r"[\"']", "", regex=True)
        .str.replace(r"\s+", " ", regex=True)
        .str.replace(r",(?=\s|$)", "", regex=True)
        .str.strip()
    )

    # Limpieza avanzada
    def cleanText(text):
        if pd.isna(text):
            return text
        text = re.sub(r"[\"']", "", text)
        text = re.sub(r",(?=\s|$)", "", text)
        text = re.sub(r"\s+", " ", text)
        text = unicodedata.normalize("NFKD", text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        return text.strip()

    df[column_name] = df[column_name].apply(cleanText)

    return df


In [1169]:
def extractEstablishmentType(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    corrections = {
        "COLEGO": "COLEGIO",
        "COLEIO": "COLEGIO",
        "INSITUTO": "INSTITUTO",
        "INSTITIUTO": "INSTITUTO",
        "INTITUTO": "INSTITUTO",
        "INSTITUTODE": "INSTITUTO",
        "INST.": "INSTITUTO",
        "'INSTITUTO": "INSTITUTO",
        "'ESCUELA": "ESCUELA",
        "'TECNOLOGICO": "TECNOLOGICO"
    }

    def classifyAndFix(name: str) -> tuple[str, str]:
        if pd.isna(name):
            return name, None

        name = name.strip().upper()
        words = name.split()
        if not words:
            return name, None

        first_word = words[0]
        corrected_type = corrections.get(first_word, first_word)
        words[0] = corrected_type
        cleaned_name = " ".join(words)

        # Clasificación especial por contenido
        if "CEEX" in cleaned_name:
            corrected_type = "CEEX"
        elif "ISEA" in cleaned_name:
            corrected_type = "HOMESCHOOL"
        elif "LOVE GUATEMALA" in cleaned_name:
            corrected_type = "CEEX"
        elif "ZETH CENTRO EDUCATIVO" in cleaned_name:
            corrected_type = "COLEGIO"
        elif "IDEAS INSTITUTO" in cleaned_name:
            corrected_type = "INSTITUTO"
        elif cleaned_name.startswith("ISA "):
            corrected_type = "ESCUELA"
        elif "IMEBI" in cleaned_name:
            corrected_type = "INSTITUTO"
        elif "GRUPO LA CEIBA" in cleaned_name:
            corrected_type = "CEEX"
        elif cleaned_name == "EL BOSQUE":
            corrected_type = "ESCUELA"
        elif "HAN AL INTERNACIONAL" in cleaned_name:
            corrected_type = "COLEGIO"
        elif "HOMESCHOOL" in cleaned_name:
            corrected_type = "HOMESCHOOL"
        elif "ENBI OXLAJUJ NO OJ" in cleaned_name:
            corrected_type = "INSTITUTO"

        # Excepciones: escuela pero debe ser instituto
        elif "INSTITUTO LA ESCUELA EN SU CASA" in cleaned_name:
            corrected_type = "INSTITUTO"
        elif "INSTITUTO DE EDUCACION A DISTANCIA LA ESCUELA EN SU CASA" in cleaned_name:
            corrected_type = "INSTITUTO"
        elif "INSTITUTO NORMAL DE PRIMARIA BILINGUE INTERCULTURAL ADS. A ESCUELA NORMAL" in cleaned_name:
            corrected_type = "INSTITUTO"
        elif "INSTITUTO DE EDUCACION DIVERSIFICADA COLEGIO EUROPEO" in cleaned_name:
            corrected_type = "INSTITUTO"

        # Si empieza con "CORPORACION"
        elif cleaned_name.startswith("CORPORACION "):
            corrected_type = "CORPORACION"

        elif corrected_type.startswith("ICA"):
            corrected_type = "COLEGIO"

        # Clasificación general por palabra clave
        elif "COLEGIO" in cleaned_name:
            corrected_type = "COLEGIO"
        elif "ESCUELA" in cleaned_name:
            corrected_type = "ESCUELA"
        elif "SCHOOL" in cleaned_name:
            corrected_type = "ESCUELA"
        elif "COLLEGE" in cleaned_name:
            corrected_type = "COLEGIO"

        # Mapeo general
        if corrected_type == "TECNOLOGICO":
            corrected_type = "TECNICO"

        return cleaned_name, corrected_type


    corrected_results = df["establecimiento"].apply(classifyAndFix)
    df["establecimiento"] = corrected_results.apply(lambda x: x[0])
    df["tipo_establecimiento"] = corrected_results.apply(lambda x: x[1])

    # Reordenar columna
    cols = df.columns.tolist()
    idx = cols.index("establecimiento")
    cols.insert(idx, cols.pop(cols.index("tipo_establecimiento")))
    df = df[cols]

    return df


In [1170]:
df = cleanTextColumn(df=df, column_name="establecimiento")
df.to_csv("./data/data_clean_v1.csv", index=False, encoding="utf-8")

In [1171]:
# reporte, df_limpio = generateDataReport(df, saveToFile=is_generate_report, filename="reporte_data_v1")

In [1172]:
df = extractEstablishmentType(df)
df.to_csv("./data/data_clean_v2.csv", index=False, encoding="utf-8")

In [1173]:
# reporte, df_limpio = generateDataReport(df, saveToFile=is_generate_report, filename="reporte_data_v2")

In [1174]:
def getUniqueEstablishmentTypes(df: pd.DataFrame) -> list:
    if "tipo_establecimiento" not in df.columns:
        raise ValueError("Column 'tipo_establecimiento' not found in DataFrame.")
    
    return df["tipo_establecimiento"].dropna().unique().tolist()

def saveTypesToFile(types: list, filepath: str):

    with open(filepath, "w", encoding="utf-8") as f:
        for tipo in types:
            f.write(f"- {tipo}\n")

In [1175]:
df = extractEstablishmentType(df)
types = getUniqueEstablishmentTypes(df)
saveTypesToFile(types, "./reportes/tipos_establecimientos.txt")

In [1176]:
def saveEstablishmentsByType(df: pd.DataFrame, tipo: str) -> None:
    import os

    tipo = tipo.strip().upper()
    filtered_df = df[df["tipo_establecimiento"] == tipo][["codigo", "establecimiento"]]
    os.makedirs("./reportes/detalle_establecimiento", exist_ok=True)
    file_name = f"./reportes/detalle_establecimiento/establecimientos_{tipo.lower()}.csv"
    filtered_df.to_csv(file_name, index=False, encoding="utf-8")


In [1177]:
def generateEstablishmentReports(df: pd.DataFrame, generateTxtFiles: bool = True) -> None:
    if generateTxtFiles:
        types = getUniqueEstablishmentTypes(df)
        saveTypesToFile(types, "./reportes/tipos_establecimientos.txt")

        for tipo in types:
            saveEstablishmentsByType(df, tipo)


In [1178]:
is_generate_txt = True
generateEstablishmentReports(df, is_generate_txt)

> Nota: Se puede modificar la bandera para omitir la creación de los reportes de establecimientos individuales.

### Código

Antes de procesar la columna `codigo`, se aplicó una limpieza previa a las columnas `departamento`, `municipio` y `departamental` para normalizar tildes, diéresis y caracteres especiales, garantizando uniformidad textual. Luego, se descompuso el valor de `codigo` siguiendo su estructura estándar (`XX-YY-ZZZZ-WW`), extrayendo tres componentes: el código de establecimiento (`ZZZZ`), el código interno (`WW`) y los códigos geográficos departamentales (`XX`) y municipales (`YY`). Estos se cruzaron con los nombres normalizados para generar una clave única `codigo_geografico`. Además, se extrajo el identificador del distrito (`YY` en `distrito`) como `codigo_distrito`, y se eliminaron las columnas redundantes: `codigo`, `departamento`, `municipio` y `distrito`.

In [1179]:
def cleanGeographicNames(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    def normalize(text):
        if pd.isna(text):
            return text
        text = str(text).strip().upper()
        text = unicodedata.normalize("NFKD", text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        text = re.sub(r"\s+", " ", text)
        return text

    for col in ["departamento", "municipio", "departamental"]:
        if col in df.columns:
            df[col] = df[col].apply(normalize)

    return df

In [1180]:
def extractGeographicCodes(df: pd.DataFrame, saveToCsv: bool = True) -> None:
    df_codes = df[["codigo", "departamento", "municipio"]].copy()

    # Separar el código original
    parts = df_codes["codigo"].str.split("-", expand=True)
    df_codes["codigo_departamento"] = parts[0]
    df_codes["codigo_municipio"] = parts[1]

    # Obtener combinaciones únicas
    df_geo = (
        df_codes[["codigo_departamento", "departamento", "codigo_municipio", "municipio"]]
        .drop_duplicates()
        .reset_index(drop=True)
    )

    # Agregar identificador único (secuencial)
    df_geo.insert(0, "codigo", df_geo.index + 1)

    if saveToCsv:
        df_geo.to_csv("./data/codigos_geograficos.csv", index=False, encoding="utf-8")

In [1181]:
df = cleanGeographicNames(df)
is_generate_geo_codes = True
extractGeographicCodes(df, is_generate_geo_codes)

> Nota: Se puede modificar la bandera para omitir la creación del csv de códigos geográficos.

In [1182]:
def enrichWithCodes(df: pd.DataFrame, geoCodesPath: str = "./data/codigos_geograficos.csv") -> pd.DataFrame:
    df = df.copy()

    parts = df["codigo"].str.split("-", expand=True)
    df.insert(0, "codigo_establecimiento", parts[2])
    df.insert(1, "codigo_interno", parts[3])
    df["codigo_distrito"] = df["distrito"].str.split("-").str[1]
    df.drop(columns=["distrito"], inplace=True)

    # Reubicar código_distrito en la posición original de 'distrito' (posición 3)
    cols = df.columns.tolist()
    cod_dist = cols.pop(cols.index("codigo_distrito"))
    cols.insert(3, cod_dist)
    df = df[cols]

    df_geo = pd.read_csv(geoCodesPath).rename(columns={"codigo": "codigo_geografico"})

    # Merge por departamento y municipio
    df = df.merge(
        df_geo[["codigo_geografico", "departamento", "municipio"]],
        how="left",
        on=["departamento", "municipio"]
    )

    col = df.pop("codigo_geografico")
    df.insert(0, "codigo_geografico", col)

    # Eliminar columnas ya integradas
    df.drop(columns=["codigo", "departamento", "municipio"], inplace=True)

    return df

In [1183]:
df = enrichWithCodes(df)
df.to_csv("./data/data_clean_v3.csv", index=False, encoding="utf-8")

In [1184]:
# reporte, df_limpio = generateDataReport(df, saveToFile=is_generate_report, filename="reporte_data_v3")

### Limpieza adicional para código geográfico

In [1185]:
def cleanAndMerge(data_df, codigos_df, is_unified):
    if is_unified:
        data_df['codigo_geografico'] = data_df['codigo_geografico'].astype(str)
        codigos_df['codigo'] = codigos_df['codigo'].astype(str)
        codigos_df['codigo_departamento'] = codigos_df['codigo_departamento'].astype(str)
        codigos_df['codigo_municipio'] = codigos_df['codigo_municipio'].astype(str)

        df_merged = data_df.merge(codigos_df, left_on='codigo_geografico', right_on='codigo', how='left')
        df_merged.drop(columns=['codigo_geografico', 'codigo'], inplace=True)

        columnas_geo = ['codigo_departamento', 'departamento', 'codigo_municipio', 'municipio']
        otras_columnas = [col for col in df_merged.columns if col not in columnas_geo]
        df_merged = df_merged[columnas_geo + otras_columnas]

        # Forzar columnas clave como texto
        for col in ['codigo_departamento', 'codigo_municipio', 'codigo_establecimiento', 'codigo_interno', 'codigo_distrito']:
            if col in df_merged.columns:
                df_merged[col] = df_merged[col].astype(str)

        df_merged = df_merged.sort_values(by='codigo_departamento')
        return df_merged
    else:
        return data_df

In [1186]:
is_unified = True
data_v3 = pd.read_csv("./data/data_clean_v3.csv", dtype={
    "codigo_geografico": str,
    "codigo_establecimiento": str,
    "codigo_interno": str,
    "codigo_distrito": str
})

codigos = pd.read_csv(
    "./data/codigos_geograficos.csv",
    dtype={
        "codigo": str,
        "codigo_departamento": str,
        "codigo_municipio": str
    }
)

data_final = cleanAndMerge(data_v3, codigos, is_unified)

# Guardar preservando texto
data_final.to_csv(
    "./data/data_clean_v4.csv",
    index=False,
    quoting=csv.QUOTE_NONNUMERIC
)

In [1187]:
# reporte, df_limpio = generateDataReport(data_final, saveToFile=is_generate_report, filename="reporte_data_v4")

>Nota: para departamento y municipo se pueda saber su departamento por el codigo_geografico dado el csv "codigos_geograficos.csv" si no se hace la unificación con el código geográfico (is_unified=false).

### Departamento



Este análisis se realizó con el objetivo de verificar si las columnas **"departamento"** y **"departamental"** contenían información redundante en el archivo `data_clean_v4.csv`. Inicialmente se asumía que ambas representaban lo mismo, y se evaluaba eliminar una por duplicidad.

Sin embargo, al aplicar la función `checkDepartamentoDifference`, se identificaron múltiples casos donde los valores difieren. Esto reveló que:

* **"departamento"** corresponde a la división político-administrativa oficial de Guatemala.
* **"departamental"** representa subdivisiones organizacionales internas utilizadas para fines de gestión operativa, especialmente en el ámbito educativo.

Como resultado, **no deben considerarse equivalentes ni eliminarse sin análisis**, ya que ambas aportan información distinta y útil para el contexto institucional.


In [1188]:

def checkDepartamentoDifference(filePath):
    # Cargar archivo
    df = pd.read_csv(filePath)

    # Filtrar donde 'departamento' y 'departamental' son distintos
    diff_df = df[df['departamento'] != df['departamental']]

    if not diff_df.empty:
        # Agrupar por combinaciones distintas y guardar en CSV
        grouped = diff_df[['departamento', 'departamental']].drop_duplicates()
        grouped.to_csv('./data/difference_departamento_departamental.csv', index=False)
        return True
    else:
        return False

In [1189]:
result = checkDepartamentoDifference('./data/data_clean_v4.csv')
print(result)

True


### Teléfono y categorías

Se real
izó una limpieza estructurada del dataset, que incluyó: normalización de nombres (`supervisor`, `director`) y campos tipográficos a mayúsculas, limpieza y estandarización de direcciones, validación de teléfonos (solo 8 dígitos), reemplazo de valores vacíos por `NaN` y conversión de columnas relevantes a tipo categórico (`status`, `sector`, `area`).

In [1190]:
df = data_final.copy()

# Supervisor y director: limpieza + mayúsculas
cols_normalizar = ['supervisor', 'director']
for col in cols_normalizar:
    df = cleanTextColumn(df, col)
    df[col] = df[col].str.upper()

# Tipográficas
cols_tipograficas = ['modalidad', 'jornada', 'plan', 'status', 'sector', 'area']
for col in cols_tipograficas:
    df = cleanTextColumn(df, col)
    df[col] = df[col].str.upper()

# Vacíos como NaN
df.replace("", np.nan, inplace=True)

# Convertir columnas a categoría
cols_categoricas = ['status', 'sector', 'area']
for col in cols_categoricas:
    df[col] = df[col].astype('category')

# Dirección: limpieza + normalización + mayúsculas
df = cleanTextColumn(df, 'direccion')
df['direccion'] = df['direccion'].str.upper()
df['direccion'] = df['direccion'].str.replace(r'\bKM\b', 'KILÓMETRO', regex=True)
df['direccion'] = df['direccion'].str.replace(r'\bZONA\b', 'ZONA', regex=True)

# Teléfono: solo números válidos de 8 dígitos
df['telefono'] = df['telefono'].astype(str).str.replace(r'\D', '', regex=True)
df['telefono'] = df['telefono'].apply(lambda x: x if len(x) == 8 else np.nan)

# Guardar dataset limpio
df.to_csv("./data/data_prev_clean.csv", index=False)
print("✅ Limpieza completada y archivo guardado como '/data/data_prev_clean.csv'")

✅ Limpieza completada y archivo guardado como '/data/data_prev_clean.csv'


## Generar CSV limpio

Durante la limpieza final, se reemplazaron nulos con valores explícitos para mantener consistencia en el dataset:

* `direccion` → `"SIN DIRECCION"`
* `telefono` → `"SIN TELEFONO"`
* `director` → `"NO REGISTRADO"`

La persona encargada del análisis podrá decidir si desea imputar, eliminar o conservar estos valores.

In [ ]:
reporte, df_limpio = generateDataReport(df, saveToFile=is_generate_report, filename="reporte_data_prev_clean")

VALOR_NULO_DIRECCION = "SIN DIRECCION"
VALOR_NULO_TELEFONO = "SIN TELEFONO"
VALOR_NULO_DIRECTOR = "NO REGISTRADO"
df_limpio['direccion'].fillna(VALOR_NULO_DIRECCION, inplace=True)
df_limpio['telefono'].fillna(VALOR_NULO_TELEFONO, inplace=True)
df_limpio['director'].fillna(VALOR_NULO_DIRECTOR, inplace=True)

df_limpio.to_csv("./data_clean.csv", index=False, encoding="utf-8")
# print("\n".join(report_lines))

Reporte generado en 'reportes\reporte_data_prev_clean.txt'


C:\Users\josue\AppData\Local\Temp\ipykernel_34856\1800642574.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_limpio['direccion'].fillna(VALOR_NULO_DIRECCION, inplace=True)
C:\Users\josue\AppData\Local\Temp\ipykernel_34856\1800642574.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as

In [1192]:
_, _ = generateDataReport(df_limpio, saveToFile=True, filename="reporte_data_clean")

Reporte generado en 'reportes\reporte_data_clean.txt'
